In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
root_path = "gdrive/My Drive/Colab Notebooks/"
yolo_folder = "PyTorch-YOLOv3"
yolo_path = os.path.join(root_path, yolo_folder)
model_def = os.path.join(yolo_path, "config/yolov3.cfg")
weights_path=os.path.join(yolo_path, "weights/yolov3.weights")
class_path=os.path.join(yolo_path, "data/coco.names")
samples_images = os.path.join(yolo_path, "data/samples/")
output_path = os.path.join(root_path, "output")

import sys
sys.path.insert(1, yolo_path)

In [0]:
pip install terminaltables

  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=aadfb6bd70058aacfb4599fe4fad47da4a6151659d1b4c718cfbfb19eca2e340
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [0]:
from PIL import Image
import time
from torch.autograd import Variable
import datetime
import torch
from models import Darknet
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.datasets import ImageFolder
from utils import utils
import numpy as np
import random

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


## Predict

In [0]:
def get_prediction_yoloV3(conf_thres=0.8,
                          nms_thres=0.4,
                          batch_size=1,
                          n_cpu=0,
                          img_size=416,
                          checkpoint_model=None):
  
  os.makedirs(os.path.join(root_path, "output"), exist_ok=True)

  # Set up model
  model = Darknet(model_def, img_size=img_size).to(device)

  if weights_path.endswith(".weights"):
    # Load darknet weights
    model.load_darknet_weights(weights_path)
  else:
    # Load checkpoint weights
    model.load_state_dict(torch.load(weights_path))

  model.eval()  # Set in evaluation mode

  dataloader = DataLoader(
    ImageFolder(samples_images, img_size=img_size),
    batch_size=batch_size,
    shuffle=False,
    num_workers=4
  )

  classes = utils.load_classes(class_path)
  Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

  imgs = []  # Stores image paths
  img_detections = []  # Stores detections for each image index

  print("\nPerforming object detection:")
  prev_time = time.time()
  for batch_i, (img_paths, input_imgs) in enumerate(dataloader):
    # Configure input
    input_imgs = Variable(input_imgs.type(Tensor))

    # Get detections
    with torch.no_grad():
        detections = model(input_imgs)
        detections = utils.non_max_suppression(detections, 
                                               conf_thres, 
                                               nms_thres)

    # Log progress
    current_time = time.time()
    inference_time = datetime.timedelta(seconds=current_time - prev_time)
    prev_time = current_time
    print("\t+ Batch %d, Inference Time: %s" % (batch_i, inference_time))

    # Save image and detections
    imgs.extend(img_paths)
    img_detections.extend(detections)

  # Bounding-box colors
  cmap = plt.get_cmap("tab20b")
  colors = [cmap(i) for i in np.linspace(0, 1, 20)]

  print("\nSaving images:")
  # Iterate through images and save plot of detections
  for img_i, (path, detections) in enumerate(zip(imgs, img_detections)):

    print("(%d) Image: '%s'" % (img_i, path))

    # Create plot
    img = np.array(Image.open(path))
    plt.figure()
    fig, ax = plt.subplots(1)
    ax.imshow(img)

    # Draw bounding boxes and labels of detections
    if detections is not None:
        # Rescale boxes to original image
        detections = utils.rescale_boxes(detections, img_size, img.shape[:2])
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        bbox_colors = random.sample(colors, n_cls_preds)
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:

            print("\t+ Label: %s, Conf: %.5f" % (classes[int(cls_pred)], cls_conf.item()))

            box_w = x2 - x1
            box_h = y2 - y1

            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            # Create a Rectangle patch
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor="none")
            # Add the bbox to the plot
            ax.add_patch(bbox)
            # Add label
            plt.text(
                x1,
                y1,
                s=classes[int(cls_pred)],
                color="white",
                verticalalignment="top",
                bbox={"color": color, "pad": 0},
            )

    # Save generated image with detections
    plt.axis("off")
    plt.gca().xaxis.set_major_locator(NullLocator())
    plt.gca().yaxis.set_major_locator(NullLocator())
    filename = path.split("/")[-1].split(".")[0]
    file_path = os.path.join(output_path, filename)
    plt.savefig(f"{file_path}.png", bbox_inches="tight", pad_inches=0.0)
    plt.plot()
    plt.show()
    plt.close()

In [0]:
get_prediction_yoloV3()

## Train custom dataset

In [0]:
yolo_path = os.path.join(root_path, yolo_folder)
config_path = os.path.join(root_path, yolo_folder, "config")
print(yolo_path)
print(config_path)

gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3
gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3/config


In [0]:
cd gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3

/content/gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3


In [0]:
cd config/

/content/gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3/config


In [0]:
!bash create_custom_model.sh 26

In [0]:
cd ..

/content/gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3


In [0]:
!python -W ignore::UserWarning train.py --model_def config/yolov3-custom.cfg --data_config config/custom.data --pretrained_weights checkpoints/yolov3_ckpt_custom_87.pth --epochs 20 --batch_size 16

Namespace(batch_size=16, checkpoint_interval=1, compute_map=False, data_config='config/custom.data', epochs=20, evaluation_interval=1, gradient_accumulations=2, img_size=416, model_def='config/yolov3-custom.cfg', multiscale_training=True, n_cpu=8, pretrained_weights='checkpoints/yolov3_ckpt_custom_87.pth')

Train path: data/custom/train.txt
Test path: data/custom/valid.txt
Total classes 26
['Ah', 'Ac', 'Kh', 'Kc', 'Qh', 'Qc', 'Jh', 'Jc', '10h', '10c', '9h', '9c', '8h', '8c', '7h', '7c', '6h', '6c', '5h', '5c', '4h', '4c', '3h', '3c', '2h', '2c']
Total classes 26
0


---- Evaluating Model ----
Detecting objects: 100% 163/163 [01:05<00:00,  3.05it/s]
Computing AP: 100% 26/26 [00:00<00:00, 1133.41it/s]
+-------+------------+---------+
| Index | Class name | AP      |
+-------+------------+---------+
| 0     | Ah         | 0.90721 |
| 1     | Ac         | 0.84524 |
| 2     | Kh         | 0.89990 |
| 3     | Kc         | 0.83123 |
| 4     | Qh         | 0.81368 |
| 5     | Qc         | 0.88

### Predict

In [0]:
!python -W ignore::UserWarning test.py --model_def config/yolov3-custom.cfg --data_config config/custom_predict.data --weights_path checkpoints/yolov3_ckpt_custom_93.pth --class_path data/classes.names 

Namespace(batch_size=8, class_path='data/classes.names', conf_thres=0.001, data_config='config/custom_predict.data', img_size=416, iou_thres=0.5, model_def='config/yolov3-custom.cfg', n_cpu=8, nms_thres=0.5, weights_path='checkpoints/yolov3_ckpt_custom_93.pth')
data/custom/predict.txt
['Ah', 'Ac', 'Kh', 'Kc', 'Qh', 'Qc', 'Jh', 'Jc', '10h', '10c', '9h', '9c', '8h', '8c', '7h', '7c', '6h', '6c', '5h', '5c', '4h', '4c', '3h', '3c', '2h', '2c']
26
Compute mAP...
Detecting objects: 100% 1/1 [00:00<00:00,  1.99it/s]
Computing AP: 100% 14/14 [00:00<00:00, 11678.65it/s]
Average Precisions:
+ Class '3' (Kc) - AP: 1.0
+ Class '5' (Qc) - AP: 1.0
+ Class '6' (Jh) - AP: 1.0
+ Class '7' (Jc) - AP: 1.0
+ Class '8' (10h) - AP: 1.0
+ Class '10' (9h) - AP: 1.0
+ Class '11' (9c) - AP: 1.0
+ Class '12' (8h) - AP: 1.0
+ Class '15' (7c) - AP: 1.0
+ Class '16' (6h) - AP: 1.0
+ Class '17' (6c) - AP: 1.0
+ Class '18' (5h) - AP: 1.0
+ Class '19' (5c) - AP: 1.0
+ Class '24' (2h) - AP: 0.6666666666666666
mAP: 0.9

In [0]:
pwd

'/content/gdrive/My Drive/Colab Notebooks/PyTorch-YOLOv3'

In [0]:
!python -W ignore::UserWarning detect.py --image_folder data/custom/predict --model_def config/yolov3-custom.cfg --weights_path checkpoints/yolov3_ckpt_custom_93.pth --class_path data/custom/classes.names

Namespace(batch_size=1, checkpoint_model=None, class_path='data/custom/classes.names', conf_thres=0.8, image_folder='data/custom/predict', img_size=416, model_def='config/yolov3-custom.cfg', n_cpu=0, nms_thres=0.4, weights_path='checkpoints/yolov3_ckpt_custom_93.pth')

Performing object detection:
	+ Batch 0, Inference Time: 0:00:01.808813
	+ Batch 1, Inference Time: 0:00:01.462330
	+ Batch 2, Inference Time: 0:00:01.511671
	+ Batch 3, Inference Time: 0:00:01.506829
	+ Batch 4, Inference Time: 0:00:01.641568
	+ Batch 5, Inference Time: 0:00:01.464748

Saving images:
(0) Image: 'data/custom/predict/IMG_20191226_214929.jpg'
	+ Label: 3c, Conf: 0.97087
	+ Label: 3c, Conf: 0.95158
(1) Image: 'data/custom/predict/IMG_20191226_214938.jpg'
	+ Label: 3c, Conf: 0.99992
	+ Label: 3c, Conf: 0.98228
	+ Label: 7c, Conf: 0.81807
	+ Label: 8c, Conf: 0.39507
	+ Label: 7c, Conf: 0.34559
	+ Label: 3c, Conf: 0.15788
	+ Label: 7h, Conf: 0.00512
(2) Image: 'data/custom/predict/IMG_20191226_215029.jpg'
	+ L